# 1. Introducción

En el siguiente ejercicio se realizará el **cálculo aproximado de la famosa cifra $\pi$** a partir de la serie que da la solución al famoso problema matemático llamado "El Problema de Basilea":

<center>$\sum_{n=1}^{\infty}\frac{1}{n^2}=\frac{\pi^2}{6}$</center>

El algoritmo está basado en la idea de que, a medida que sumemos más símbolos de la serie, más preciso será el valor calculado de $\pi$ a la hora de despejar la ecuación [1].

La idea principal es mostrar la perfomance de un algoritmo que puede ser paralelizado con respecto a su resolución secuencial, para ello, implementaremos la interfaz **OpenMP**.

En este aspecto, se implementó la directiva del compilador **#pragma omp parallel for reduction (...)** para que se paralelice la sección del código donde se aplica el algoritmo. Cada ciclo del for será atendido por hilo. También se aplica la directiva de reducción para que, una vez finalizado cada hilo, se aplique a la variable compartida el valor resultante de la operación a través de una operación especificada (en nuestro caso, una suma).

Si bien se utiliza Python para creación del archivo, el programa en sí está desarrollado en C++, donde su compilación y ejecución se realiza a través de comandos tipo UNIX.

---
# 2. Armado del ambiente


No es necesaria alguna preparación previa para el ejercicio.

---
# 3. Desarrollo

### Código en C

In [ ]:
code = """
#include <iostream>
#include <vector>
#include <cstdlib>
#include <sys/time.h>
#include <omp.h>    // Cabecera OpenMP
#include <cmath>     // Impresión de números por terminal
#include <fstream>   // Para elevar a potencias y hacer raíces

// --- Macros para medir el tiempo
static double dHashTiempoHistory[3];
static struct timeval tv;

#define TIEMPO_INI(h)      \
   gettimeofday(&tv, NULL);   \
   dHashTiempoHistory[h] = tv.tv_sec + tv.tv_usec / 1000000.0;
#define TIEMPO_FIN(h)      \
   gettimeofday(&tv, NULL);   \
   dHashTiempoHistory[h] = ((tv.tv_sec + tv.tv_usec / 1000000.0) - dHashTiempoHistory[h]) * 1000; // milisegundos
#define TIEMPO_GET(h) dHashTiempoHistory[ h ]

#define HTH_TOTAL       1
#define HTH_BASILEA_SEC 2
#define HTH_BASILEA_OMP 3
// --- Fin macros para medir el tiempo


int main(int argc, char* argv[]) { 
    int i;
    double sum, pi_s, pi_omp;

    // Capturamos el tiempo inicial
    TIEMPO_INI(HTH_TOTAL)

    // Validamos parámetros y los obtenemos
    if(argc != 2){
        std::cerr<< "Error en los parámetros, debe indicar únicamente la cantidad de elementos a sumar." << argc << std::endl;
        exit(-1);
    }
    int cant_terminos = atoi(argv[1]);
    
    // --- Problema de Basilea de forma secuencial 
    
    // Capturamos el tiempo inicial de la forma secuencial
    TIEMPO_INI(HTH_BASILEA_SEC)

    // Calculamos Pi
    sum = 0;
    for (i = 0; i < cant_terminos; i++){
        sum += 1/pow(i + 1, 2);   
    }
    pi_s = sqrt(6 * sum);

    // Capturamos el tiempo total de la forma secuencial
    TIEMPO_FIN(HTH_BASILEA_SEC)

    // --- Fin Problema de Basilea de forma secuencial
    
    // --- Problema de Basilea con OpenMP 

    // Capturamos el tiempo inicial con OpenMP
    TIEMPO_INI(HTH_BASILEA_OMP)

    // Calculamos Pi
    sum = 0;
    #pragma omp parallel for reduction(+: sum)
    for (i = 0; i < cant_terminos; i++){
        sum += 1/pow(i + 1, 2);   
    }
    pi_omp = sqrt(6 * sum); 
    
    // Capturamos el tiempo total con OpenMP
    TIEMPO_FIN(HTH_BASILEA_OMP)

    // Capturamos el tiempo total del ejercicio
    TIEMPO_FIN( HTH_TOTAL )

    std::cout<<std::endl;
    std::cout<<"--- Valores de Pi ---" <<std::endl;
    std::cout.precision(13);
    std::cout<<"Pi Secuencial: "<<pi_s<<" [ms]"<<std::endl;
    std::cout.precision(13);
    std::cout<<"Pi MPI       : "<<pi_omp<<" [ms]"<<std::endl;
    std::cout<<std::endl;

    std::cout<<"--- Métricas reales ---"<<std::endl;
    std::cout<<"Tiempo total     : "<<TIEMPO_GET(HTH_TOTAL   )<<" [ms]"<<std::endl;
    std::cout<<"Tiempo secuencial: "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" [ms]"<<std::endl;
    std::cout<<"Tiempo OpenMP    : "<<TIEMPO_GET(HTH_BASILEA_OMP)<<" [ms]"<<std::endl;
    std::cout<<"SpeedUp          : (tiempo secuencial/tiempo paralelo): "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" / "<<TIEMPO_GET(HTH_BASILEA_OMP)<<" = "<<TIEMPO_GET(HTH_BASILEA_SEC)/TIEMPO_GET(HTH_BASILEA_OMP)<<std::endl;
    std::cout<<"Eficiencia       : SpeedUp / Nro. procesadores        : "<<TIEMPO_GET(HTH_BASILEA_SEC)/TIEMPO_GET(HTH_BASILEA_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_BASILEA_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP))<<std::endl;
    std::cout<<"Coste sec        : Nro. procesadores * tiempo         : "<<1<<" * "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" = "<<TIEMPO_GET(HTH_BASILEA_SEC)<<std::endl;
    std::cout<<"Coste OMP        : Nro. procesadores * tiempo         : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_BASILEA_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP)<<std::endl;
    std::cout<<"Funcion overhead : Coste OMP - tiempo secuencial      : "<<omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP)<<" - "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP))-TIEMPO_GET(HTH_BASILEA_SEC)<<std::endl;
    std::cout<<std::endl;

    std::cout<<"--- Métricas ideales ---"<<std::endl;
    TIEMPO_GET(HTH_BASILEA_OMP) = TIEMPO_GET(HTH_BASILEA_SEC) / 2;
    std::cout<<"Tiempo secuencial: "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" [ms]"<<std::endl;
    std::cout<<"Tiempo OpenMP    : "<<TIEMPO_GET(HTH_BASILEA_OMP)<<" [ms]"<<std::endl;
    std::cout<<"SpeedUp          : (tiempo secuencial / tiempo paralelo): "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" / "<<TIEMPO_GET(HTH_BASILEA_OMP)<<" = "<<TIEMPO_GET(HTH_BASILEA_SEC)/TIEMPO_GET(HTH_BASILEA_OMP)<<std::endl;
    std::cout<<"Eficiencia       : SpeedUp / Nro. procesadores          : "<<TIEMPO_GET(HTH_BASILEA_SEC)/TIEMPO_GET(HTH_BASILEA_OMP)<<" / "<<omp_get_num_procs()<<" = "<<TIEMPO_GET(HTH_BASILEA_SEC)/(omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP))<<std::endl;
    std::cout<<"Coste sec        : Nro. procesadores * tiempo           : "<<1<<" * "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" = "<<TIEMPO_GET(HTH_BASILEA_SEC)<<std::endl;
    std::cout<<"Coste OMP        : Nro. procesadores * tiempo           : "<<omp_get_num_procs()<<" * "<<TIEMPO_GET(HTH_BASILEA_OMP)<<" = "<<omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP)<<std::endl;
    std::cout<<"Funcion overhead : Coste OMP - tiempo secuencial        : "<<omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP)<<" - "<<TIEMPO_GET(HTH_BASILEA_SEC)<<" = "<<(omp_get_num_procs()*TIEMPO_GET(HTH_BASILEA_OMP))-TIEMPO_GET(HTH_BASILEA_SEC)<<std::endl;
}
"""
text_file = open("basilea.cpp", "w")
text_file.write(code)
text_file.close()

### Compilación del código C

In [ ]:
!g++ -o basilea -fopenmp basilea.cpp

### Ejecución del programa

In [ ]:
%env OMP_NUM_THREADS=16
!./basilea 500000

### Medidas de prestaciones en algoritmos paralelos
Las tecnicas de HPC buscan reducir los tiempos de ejecución, el tiempo como meida, no alcanza. Dos algoritmos pueden ejecutar en el mismo tiempo, pero uno de ellos usa menos procesadores [2][3]. 


#### SpeedUp
Referencia a la ganacia de velocidad que se consigue con un algoritmo paralelo, al resolver el mismo problema con respecto al algoritmo secuencial.

<center>$ SpeedUp = TiempoSecuencial  /  TiempoParalelo $</center>



#### Eficiencia
La eficiencia normaliza el valor del SpeedUp, al dividirlo por la cantidad de procesadores que se utilizaron para alcanzar la ganacia en velocidad. Dando la idea de la porción de tiempo que los procesadores se dedican al trabajo útil.

<center>$ Eficiencia = SpeedUp  /Nro. procesadores $</center>


#### Coste
El coste de un algoritmo paralelo representa el tiempo realizado por todo el sistema en la resoluciòn del problema.

<center>$ coste = Nro. procesadores * Tiempo algoritmo$</center>


#### Función Overhead
Es la diferencia entre el Coste y el tiempo secuencial. Mientras mayor es la función overhead, peor es el comportamiento del algoritmo paralelo.
<center>$ Overhead = Coste  /  TiempoSecuencial $</center>


# 4. Tabla de pasos

# 5. Conclusiones

---
# 6. Bibliografía

[1] Euler y El Problema de Basilea: [PDF](http://www2.caminos.upm.es/Departamentos/matematicas/revistapm/revista_impresa/vol_V_num_1/his_mat_euler_basilea.pdf)

[2] F. Almeida, D. Gimenéz, A. Vidal - Introducción a la programación paralela - 2008 - Editorial Parafino.

[3] D. Jiménez-González - Introducción a las arquitecturas paralelas. [PDF](http://so-unlam.com.ar/material-clase/HPC/Arquitecturas_de_computadores_avanzadas_(Modulo_1).pdf)